# Used Car Advertisement Exploratory Data Analysis

In this experiment, we take a look at a dataset of car sales advertisements. We're interested in factors such as the distribution of price based on the vehicles brand, and when people may start thinking about selling their car by looking at the odometer readings of the vehicles. We are also going to incorporate this data into an interactive web application so that users can make their own comparisons.

In [55]:
import pandas as pd
import plotly.express as px
import streamlit as st


### Data Cleaning

In [ ]:
data = pd.read_csv('vehicles_us.csv')
print(data.info(show_counts=True))
print(data.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB
None
   price  model_year           model  condition  cylinders fuel  odometer  \
0   9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1  25500         NaN     

Out of the columns with missing variables, model_year and odometer might be worth taking a look at.

When making plots with price, I noticed that there are price values that are extremely low, such as new vehicles supposedly selling for less than 500 dollars, which is odd. I'll take a look at that.

In [57]:
# filter for vehicles priced more than 500 USD
normal_price = data[data['price'] > 500]

# checking the cases where vehicles are priced less than 500
price_is_low = data[data['price'] < 500]
price_is_low.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
31953,277,2014.0,chevrolet impala,excellent,6.0,gas,139890.0,automatic,sedan,black,NaN,2018-10-10,19
11318,1,2017.0,toyota corolla,excellent,4.0,gas,27295.0,automatic,sedan,white,1.0,2019-01-21,17
14639,1,2018.0,chevrolet suburban,excellent,8.0,gas,16931.0,automatic,SUV,black,1.0,2018-05-21,4
12184,1,2018.0,gmc acadia,excellent,6.0,gas,23373.0,automatic,SUV,NaN,1.0,2018-09-10,20
26538,1,2018.0,honda civic,excellent,4.0,gas,31829.0,manual,sedan,white,1.0,2018-10-30,17
45056,1,NaN,chevrolet camaro,excellent,10.0,gas,36927.0,other,coupe,silver,1.0,2018-09-04,31
14006,1,2018.0,dodge charger,excellent,10.0,gas,NaN,other,sedan,custom,1.0,2018-08-04,23
13105,1,2018.0,dodge charger,excellent,8.0,gas,14245.0,automatic,sedan,NaN,1.0,2018-05-30,33
11534,1,2014.0,toyota tundra,excellent,8.0,gas,89606.0,automatic,truck,NaN,1.0,2018-11-22,23
30783,1,2019.0,nissan maxima,excellent,6.0,gas,18004.0,automatic,sedan,custom,NaN,2019-03-27,12


These values are likely placeholders in the case that the price is 1. I'm guessing that the other sub 500 vehicles had their values input incorrectly. For example, zeroes may be missing. Either that or the condition isn't input correctly. Luckily there aren't a lot of them. I'll leave these out and it shouldn't affect analysis.

Here I add a manufacturer column to make it easier to compare different brands.

In [58]:
# Add manufacturer column
data['manufacturer'] = data['model'].apply(lambda x: x.split()[0])

## Duplicates Check

In [59]:
print(data.duplicated().value_counts())
print(data.duplicated().sum())

False    51525
Name: count, dtype: int64
0


Doesn't look like there are any duplicate entries.

I'm interested in seeing how the top 3 manufacturers compare to the rest since they account for more than half of the entries in the dataset.

In [60]:
# Get the top 3 manufacturers
top_manufacturers = data['manufacturer'].value_counts().head(3).index.tolist()

# Filter for only top 3 manufacturers
filtered_top_data = data[data['manufacturer'].isin(top_manufacturers)]

# Filter for all other manufacturers
filtered_not_data = data[~data['manufacturer'].isin(top_manufacturers)]

data['manufacturer'].value_counts()

manufacturer
ford             12672
chevrolet        10611
toyota            5445
honda             3485
ram               3316
jeep              3281
nissan            3208
gmc               2378
subaru            1272
dodge             1255
hyundai           1173
volkswagen         869
chrysler           838
kia                585
cadillac           322
buick              271
bmw                267
acura              236
mercedes-benz       41
Name: count, dtype: int64

## Missing Values

Here I'll explore the missing values in the dataset. We'll start with categorical values is_4wd and paint_color.

In [61]:
# Filling in NaN values with 0 to show that they are not 4WD
data.fillna({'is_4wd': 0}, inplace=True)

# Filling in NaN values with Unknown
data.fillna({'paint_color': 'Unknown'}, inplace=True)


Next, we'll take a look at numerical categories model_year, odometer, and cylinders.

### Model Year Values

Here, we assume that the model year category is related to the model. We group models together and assign the median model year to fill in the missing values.

In [62]:

data['model_year'] = data['model_year'].fillna(data.groupby(['model'])['model_year'].transform('median'))
data['model_year'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 51525 entries, 0 to 51524
Series name: model_year
Non-Null Count  Dtype  
--------------  -----  
51525 non-null  float64
dtypes: float64(1)
memory usage: 402.7 KB


### Odometer Values

Here, we'll also use the median for odometer values, grouped by the model. We'll assume that the same models handles odometer readings similarly.

In [63]:
data['odometer'] = data['odometer'].fillna(data.groupby(['model'])['odometer'].transform('median'))
data['odometer'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 51525 entries, 0 to 51524
Series name: odometer
Non-Null Count  Dtype  
--------------  -----  
51484 non-null  float64
dtypes: float64(1)
memory usage: 402.7 KB


### Cylinder Values

Lastly, we'll fill in the missing cylinder values. We're assuming that the same models will have the same number of cylinders, so we use the median to fill them in.

In [64]:
data['cylinders'] = data['cylinders'].fillna(data.groupby(['model'])['cylinders'].transform('median'))
data['cylinders'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 51525 entries, 0 to 51524
Series name: cylinders
Non-Null Count  Dtype  
--------------  -----  
51525 non-null  float64
dtypes: float64(1)
memory usage: 402.7 KB


I'll take a look at the distribution of the vehicle prices between the 3 manufacturers that show up the most often in the dataset compared to the rest of the manufacturers. I'm guessing that the top 3 will have slightly higher prices show up more compared to rest of the dataset.

In [65]:
price_top_3 = px.histogram(
                            filtered_top_data,
                            x='price',
                            nbins=60,
                            title='Price Distribution of Ford, Toyota, and Chevrolet Vehicles',
                            labels={
                                'price': 'Price (USD)',
                                'count': 'Frequency'
                            })

price_top_3.update_xaxes(range=[0, 60000]) # Set x-axis range for clarity
price_top_3.update_layout(yaxis_title='Frequency')
price_top_3


In [66]:
price_other = px.histogram(
                            filtered_not_data,
                            x='price',
                            nbins=100,
                            title='Price Distribution of All Other Manufacturers',
                            labels={
                                'price': 'Price (USD)',
                                'count': 'Frequency'
                            })

price_other.update_xaxes(range=[0, 60000]) # Set x-axis range for clarity
price_other.update_layout(yaxis_title='Frequency')
price_other

Surprisingly, there isn't too much of a difference between the top 3 brands and the rest of the brands. Most notably, there are more frequent values in the 25-30k range in the top 3 brands compared to the rest.

I'm also curious in the relationship between a vehicles odometer reading and it's price. Since there are an overwhelmingly large amount of values, we'll look at this with a heatmap.

In [67]:
# 
price_odometer = px.density_heatmap(
                                    data.query('2 <= price <= 60000 & odometer <= 200000'),  # Filter outliers
                                    x='odometer',
                                    y='price',
                                    nbinsx=25,
                                    nbinsy=25,
                                    color_continuous_scale='Viridis',
                                    labels={
                                        'odometer': 'Odometer (miles)',
                                        'price': 'Price (USD)'},
                                    title='Density of Odometer vs Price'
)

price_odometer

## Conclusion

From this analysis, we see that the price distribution between the top 3 manufacturers and that of the rest of the dataset doesn't seem to differ very much.
Overall, the distribution has a peak of values in the 5-10k price range with a positive skew. In other words, most of the cars available in the dataset are being sold for more than 10,000 USD.

We see that same peak of value in the density heatmap, where most of the vehicles sold in the 5-10k price range will have around 90,000 or more miles on its odometer.
Looks like there's an inverse relationship between how a car is priced and the amount of miles on it's odometer. In general, people may consider selling their car once the odometer hits around 100,000 miles for about $5,000-$15,000.